# **Data Preparation**

## **Introduzione**
In questa fase, ci concentriamo sulla preparazione del dataset per garantire che sia pronto per la modellazione. Le attività principali includono la pulizia dei dati, la creazione di nuove feature, la standardizzazione e la selezione delle variabili più rilevanti.



## **1. Caricamento del Dataset**
Iniziamo caricando il dataset `cardio_train.csv` per esaminarne il contenuto e le caratteristiche principali.


In [25]:
import pandas as pd

# Caricamento del dataset
data_path = 'cardio_train.csv'
data = pd.read_csv(data_path, sep=';')

# Visualizza le prime righe del dataset
data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


## **2. Pulizia dei Dati**
Gli outlier possono distorcere le analisi. Rimuoviamo valori anomali relativi alla pressione arteriosa, altezza e peso.

In [26]:
# Rimuovi valori irrealistici per pressione, altezza e peso
data = data[(data['ap_hi'] >= 90) & (data['ap_hi'] <= 180)]
data = data[(data['ap_lo'] >= 60) & (data['ap_lo'] <= 120)]
data = data[(data['height'] >= 120) & (data['height'] <= 200)]
data = data[(data['weight'] >= 30) & (data['weight'] <= 200)]

## **3. Creazione di Nuove Feature**
Indice di Massa Corporea (BMI)
Il **BMI** è una metrica chiave per valutare il rischio cardiovascolare.

In [27]:
# Calcolo del BMI
data['BMI'] = data['weight'] / ((data['height'] / 100) ** 2)

**Rapporto Sistole-Diastole**

In [28]:
# Calcolo del rapporto sistole-diastole
data['bp_ratio'] = data['ap_hi'] / data['ap_lo']

**Conversione età da giorni a anni**

In [29]:
# Conversione dell'età
data['age_years'] = (data['age'] / 365).astype(int)

## 4. **Standardizzazione delle variabili**

In [30]:
from sklearn.preprocessing import StandardScaler
import joblib

# Standardizzazione delle feature numeriche
scaler = StandardScaler()
numerical_features = ['age_years', 'BMI', 'ap_hi', 'ap_lo', 'bp_ratio']
data[numerical_features] = scaler.fit_transform(data[numerical_features])
joblib.dump(scaler, 'scaler.pkl')
print("Scaler salvato correttamente.")
print("Mean:", scaler.mean_)
print("Variance:", scaler.var_)
joblib.dump(scaler, 'scaler.pkl')


Scaler salvato correttamente.
Mean: [ 52.82406198  27.44793095 126.43749725  81.26471408   1.55987263]
Variance: [4.58249791e+01 2.74759148e+01 2.54732087e+02 8.35725918e+01
 2.01673990e-02]


['scaler.pkl']

## 6. **Selezione delle Feature**

Determinazione delle variabili più rilevanti usando un modello Random Forest.

In [19]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Rimuovi le feature inutili o ridondanti
data = data.drop(columns=['id', 'age', 'weight', 'height'])

# Dividi il dataset in feature e target
X = data.drop('cardio', axis=1)
y = data['cardio']

# Allenamento di Random Forest per l'importanza delle feature
rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)

# Visualizza l'importanza delle feature
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values(by='importance', ascending=False)

print(feature_importances)


        feature  importance
8           BMI    0.474641
10    age_years    0.155305
1         ap_hi    0.132565
2         ap_lo    0.079673
9      bp_ratio    0.047297
3   cholesterol    0.040622
4          gluc    0.019120
0        gender    0.017974
7        active    0.014650
5         smoke    0.009851
6          alco    0.008301


In [20]:
from sklearn.inspection import permutation_importance

# Calcola l'importanza tramite permutazione
perm_importance = permutation_importance(rf, X, y, scoring='accuracy', random_state=42)

# Organizza i risultati in un DataFrame
perm_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': perm_importance.importances_mean
}).sort_values(by='importance', ascending=False)

print(perm_importances)

        feature  importance
10    age_years    0.278108
1         ap_hi    0.272635
8           BMI    0.268670
3   cholesterol    0.136134
0        gender    0.111271
2         ap_lo    0.108911
9      bp_ratio    0.102035
7        active    0.084970
4          gluc    0.067910
5         smoke    0.033538
6          alco    0.019683


## **Analisi delle Feature: Confronto tra Random Forest e Permutation Importance**

### **1. Risultati Originali (Random Forest Importance)**
La Random Forest calcola l'importanza delle feature in base al decremento dell'impurità del nodo (Gini Impurity) aggregato su tutti gli alberi. Ecco i risultati iniziali:

| **Feature**     | **Importance** |
|------------------|----------------|
| BMI             | 0.4746         |
| age_years       | 0.1553         |
| ap_hi           | 0.1326         |
| ap_lo           | 0.0797         |
| bp_ratio        | 0.0473         |
| cholesterol     | 0.0406         |
| gluc            | 0.0191         |
| gender          | 0.0179         |
| active          | 0.0147         |
| smoke           | 0.0099         |
| alco            | 0.0083         |

- **Osservazioni**:
  - **BMI** domina come feature più importante.
  - **smoke** e **alco** sono classificate come meno influenti, probabilmente a causa dello squilibrio nei dati (pochi valori "1").

### **2. Risultati con Permutation Importance**
La Permutation Importance valuta l'importanza delle feature mescolandole casualmente e misurando l'impatto sulla performance del modello. Risultati:

| **Feature**     | **Importance** |
|------------------|----------------|
| age_years       | 0.2781         |
| ap_hi           | 0.2726         |
| BMI             | 0.2687         |
| cholesterol     | 0.1361         |
| gender          | 0.1113         |
| ap_lo           | 0.1089         |
| bp_ratio        | 0.1020         |
| active          | 0.0850         |
| gluc            | 0.0679         |
| smoke           | 0.0335         |
| alco            | 0.0197         |

- **Osservazioni**:
  - **age_years**, **ap_hi**, e **BMI** sono comparabili in importanza, suggerendo che il loro contributo predittivo è simile.
  - **smoke** e **alco** hanno ora valori leggermente più alti rispetto alla Random Forest, indicando che la loro bassa importanza precedente era influenzata dallo squilibrio nei dati.

### **3. Perché Permutation Importance è Fondamentale?**
La Permutation Importance è fondamentale perché:
1. **Non è influenzata dagli squilibri nei dati**: Valuta l'impatto effettivo di ogni feature sulla predizione, indipendentemente dalla frequenza dei valori.
2. **Considera l'interazione tra le feature**: Diversamente dalla Random Forest, che può sottostimare l'importanza delle feature che interagiscono con altre.
3. **Fornisce un'interpretazione più robusta**: Mescolando una feature alla volta, si osserva il cambiamento diretto nella performance del modello, fornendo una visione chiara del suo contributo.

### **4. Conclusioni**
- **BMI**, **age_years**, e **ap_hi** sono fondamentali in entrambi i metodi, confermando la loro rilevanza.
- **smoke** e **alco**, sebbene meno influenti, hanno un ruolo che potrebbe essere sottovalutato dalla Random Forest.
- La Permutation Importance è particolarmente utile quando si lavora con dataset sbilanciati o con feature che interagiscono fortemente con altre.

**Raccomandazione**: Considerare entrambe le tecniche per un'analisi completa dell'importanza delle feature. Includere feature come **smoke** e **alco** potrebbe essere giustificato per ottenere una visione più ampia del problema.


In [21]:
# Esporta il dataset preparato
prepared_file_path = 'prepared_cardio_train.csv'

# Salva il dataset preprocessato in formato CSV
data.to_csv(prepared_file_path, index=False)

print(f"Dataset esportato come: {prepared_file_path}")


Dataset esportato come: prepared_cardio_train.csv
